In [1]:
from ldtp import *
import zeep
from astropy.time import Time
import sys
from os import remove, path, getcwd
from shutil import copyfile
from struct import unpack_from
import numpy as np
from re import finditer
from prettytable import PrettyTable

In [2]:
def get_date(filename):
    # Reading binary data
    with open(filename, "rb") as f:
        bin_data = f.read()

    # Searching datetime start byte
    date_key = '\xe0.\x00\x00\x0b'
    matches = []
    for match in finditer(date_key, bin_data):
        matches.append(match.span())
    date_offset = matches[-1][0] - 8
    
    # Unpacking datetime
    time_values = {}
    hours, days = unpack_from("<II", bin_data, date_offset)
    secs = days * 45.0 * 60.0 + hours / (4294967295.0/45.0) * 60.0
    time_values["UNIX"] = secs - 2938117104000.0

    # Convert to ISO time format
    t = Time(time_values["UNIX"], format="unix")
    t.format = "iso"
    t.out_subfmt = "date"
    return t.value

In [3]:
def get_buttons(window):
    objects = getobjectlist(window)
    tbar_idx = objects.index("tbar0")
    buttons = objects[tbar_idx + 5 : tbar_idx + 8]
    buttons.append(objects[tbar_idx + 11])
    return buttons

In [4]:
def g9_proc(filename, wzz_1, wzz_2):
    
    g9_bin = "C:\\Program Files\\Micro-g LaCoste Inc\\bin\\g9.exe"
    wdw = "*Micro-g*"
    results = {}
    
    results["date"] = get_date(filename)
    
    if not guiexist(wdw):
        launchapp(g9_bin)
    
    dlgOpenFiles = u'frm\u041e\u0442\u043a\u0440\u044b\u0442\u044c'
    txtFileName = u'txt\u0418\u043c\u044f\u0444\u0430\u0439\u043b\u0430'
    btnOpenFile = u'btn\u041e\u0442\u043a\u0440\u044b\u0442\u044c2'

    if guiexist('frmState'):
        selectmenuitem(wdw, "mnuProject;mnuClose")

    selectmenuitem(wdw, "mnuProject;mnuOpen Project...")
    wait(1)
    settextvalue(dlgOpenFiles, txtFileName, filename)
    click(dlgOpenFiles, btnOpenFile)
    wait(1)
    
    btnSetup, btnGo, btnStop, btnAbout = get_buttons(wdw)
    
    click(wdw, btnSetup) 
    wait(1)
    
    results["site_name"] = gettextvalue('frmSetup', 'txt0')
    
    # Set Control Tab
    selecttab('frmSetup', 'ptl0', 'ptabControl')
        
    # Turn off Corrections
    click('frmSetup', 'btnSetup4')
    for button in ['chkSelfAttraction',
                     'chkBarometricPressure',
                     'chkPolarMotion',
                     'chkReferenceXo',
                     'chkTransferHeight',
                     'chkDiffraction']:
        uncheck('frmCorrectionsSetup', button)
    click('frmCorrectionsSetup', 'btnOK')

    # Turn off Tidal Correction
    comboselect('frmSetup', 'cbo1', "None")

    # Maximize Reject Sigma
    settextvalue('frmSetup', 'txt6', "50")

    # Set Uncertainties to zeros
    click('frmSetup', 'btnSetup2')
    for i in range(12):
        settextvalue('frmUncertainties', 'txt' + str(i), "0")
    click('frmUncertainties', 'btnOK')
    
    # Push OK
    click("frmSetup", u'btn\u041e\u041a')
    wait(1)
    
    gravity = []
    for wzz in [str(wzz_1), str(wzz_2)]:
        click(wdw, btnAbout)
        click('frmAboutMGL', 'btnOK')
        
        click(wdw, btnSetup) 
        wait(1)
    
        # Set gradient
        settextvalue("frmSetup", 'txt6', wzz)
        
        # Push OK
        click("frmSetup", u'btn\u041e\u041a')
        wait(1)
    
        # Start processing
        click(wdw, btnGo)
        wait(1)

        if guiexist('frmOverrideDialog'):
            click('frmOverrideDialog', 'btnYes')

        # Waiting till processing ends
        total_sets = int(gettextvalue('frmState', 'txtAcquisition'))
        wait(5)
        while int(gettextvalue('frmState', 'txt32')) != total_sets:
            wait(3)
        wait(5)

        gravity.append(float(gettextvalue('frmState', 'txt42')))

    selectmenuitem(wdw, "mnuProject;mnuClose")
    
    # Calculating h_mod
    h_mod = (gravity[0] - gravity[1]) / (wzz_1 - wzz_2)
    
    results["gravity"] = gravity
    results["h_mod"] = h_mod
    
    return results

In [5]:
def main():
    projects_path = "E:\\gProjects_mod"
    outfile =  "report_h_mod"
    wzz1 = -3.0
    wzz2 = -4.0
    
    # Make list with all existing projects
    projects = []
    for point in os.listdir(projects_path):
        point_path = path.join(projects_path, point, "ag")
        if os.path.isdir(point_path):
            for date in os.listdir(point_path):
                date_path = path.join(point_path, date, "fg5110\\data")
                if os.path.isdir(date_path):
                    projects.append(date_path)
    
    # Pretty table init
    x = PrettyTable()
    x.field_names = ["site", "date", "azimuth", "g_1, uGal", 
                     "g_2, uGal", "h_mod, cm"]
    
    rslt = []
    # Process all projects
    for project in projects:
        for az in ["north", "south"]:
            az_file = path.join(project, az, az + ".fg5")
            if path.isfile(az_file):
                # Processing gravity with vertical gradients -3 and -4
                project_h_mod = path.splitext(az_file)[0] + "_h_mod.fg5"
                copyfile(az_file, project_h_mod)
                az_results = g9_proc(project_h_mod, wzz1, wzz2)
                rslt.append(az_results["h_mod"])
                
                print az_results["h_mod"]

                x.add_row([az_results["site_name"], az_results["date"],
                           az, "{:.2f}".format(az_results["gravity"][0]), 
                           "{:.2f}".format(az_results["gravity"][1]), 
                           "{:.2f}".format(az_results["h_mod"])])

                # Cleaning folder
                for ext in [".fg5", ".project.txt", ".set.txt"]:
                    rm_name = path.splitext(project_h_mod)[0] + ext
                    try:
                        remove(rm_name)
                    except:
                        print("File %s doesn't exists" % path.basename(rm_name))
            else:
                print("Project file for %s azimuth not found" % az)
                continue
   
    nt = Time.now()
    nt.format = "iso"
    nt.out_subfmt = "date"
    
    with open(outfile, "w") as f:
        
        f.write("Calculation of h_mod - height which is\
        \ndependent on laser modulation frequency\
        \n\nh_mod = (g_1 - g_2) / (wzz_1 - wzz_2)\n\n")
        
        f.write("g9 Control Tab Settings:\
        \n- All checkboxes in Corrections Setup\
        \n  are unchecked\
        \n- Tidal Correction box set to 'None'\
        \n- Reject Sigma set to '50'\
        \n- All Uncertainties values set to '0'\n\n")

        f.write(("Date:  {0}\nProjects Processed:  {1}\
        \nwzz_1:  {2:.1f} \xb5Gal/cm\nwzz_2:  {3:.1f} \xb5Gal/cm\
        \nh_mod_mean:  {4:.2f} cm\
        \nh_mod_sd:  {5:.3f} cm\n\n").format(nt.value, len(rslt),
        wzz1, wzz2, np.mean(rslt), np.std(rslt)))
        
        f.write(x.get_string())

In [6]:
if __name__ == "__main__":
    main()

7.07999992371
7.06999993324
7.07000005245
7.07000005245
7.07000005245
7.07000005245
7.08000004292
7.08000004292
7.06999993324
7.08000004292
7.08000004292
7.07000005245
7.08000004292
7.07999992371
7.07000005245
7.07000005245
7.07000005245
7.06999993324
7.09000003338
7.07000005245
7.09000003338
7.07000005245
